In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import io
import logging
import pprint

import pandas as pd

import core.config as cconfig
import optimizer.single_period_optimization as osipeopt
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hprint as hprint

In [3]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

# _LOG.info("%s", henv.get_system_signature()[0])

INFO: > cmd='/venv/lib/python3.9/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-129cf3cb-706b-413c-8cbc-be607e2c7eee.json'


In [4]:
hprint.config_notebook()

ModuleNotFoundError: No module named 'matplotlib'

# Build forecast dataframe

In [7]:
idx = [1, 2, 3]

In [8]:
sigma = pd.Series(index=idx, data=[0.05, 0.07, 0.08], name="volatility")
holdings = pd.Series(index=idx, data=[1000, 1500, -500], name="position")
predictions = pd.Series(index=idx, data=[0.05, 0.09, 0.03], name="prediction")

In [9]:
df = pd.concat([holdings, predictions, sigma], axis=1)

In [10]:
df.index.name = "asset_id"

In [11]:
df

,position,prediction,volatility
asset_id,,,
1,1000,0.05,0.05
2,1500,0.09,0.07
3,-500,0.03,0.08


In [12]:
df = df.reset_index()

In [13]:
df

,asset_id,position,prediction,volatility
0,1,1000,0.05,0.05
1,2,1500,0.09,0.07
2,3,-500,0.03,0.08


# Build optimizer config

In [14]:
dict_ = {
        "volatility_penalty": 0.75,
        "dollar_neutrality_penalty": 0.1,
        "turnover_penalty": 0.0,
        "target_gmv": 3000,
        "target_gmv_upper_bound_multiple": 1.01,
    }

spo_config = cconfig.get_config_from_nested_dict(dict_)

# Optimize

In [15]:
spo = osipeopt.SinglePeriodOptimizer(spo_config, df)

In [16]:
opt_results = spo.optimize()
display(opt_results.round(3))

INFO  `optimal_value`=0.03


,target_positions,target_notional_trades,target_weights,target_weight_diffs
asset_id,,,,
1,-2.445,-1002.445,-0.001,-0.334
2,1523.654,23.654,0.508,0.008
3,-1508.575,-1008.575,-0.503,-0.336


In [17]:
spo.compute_stats(opt_results)

,notional,percentage
gross_volume,2034.67360,0.678225
net_volume,-1987.36479,-0.662455
gmv,3034.67360,1.011558
nmv,12.63521,0.004212


In [31]:
assert 0

AssertionError: 

In [32]:
df = pd.DataFrame(
    [[0.000843, -0.001103, 0.001103, 24000],
     [-0.000177, -0.000344, 0.000344, -17000],
     [-0.000238, -0.000130, 0.000130, -90000]],
    [101, 201, 301],
    ["prediction", "returns", "volatility", "position"],
)
df.index.name = "asset_id"
df = df.reset_index()